# Sofia Air Case - Team BG-USA

#### Team BG-USA Team Members:
- Sergey Vichev (serjvichev@gmail.com) datachat: sergeyvi4ev
- 

In [1]:
import pandas as pd
import numpy as np
import s3fs
import scipy
from datetime import datetime
import matplotlib as plt
import seaborn as sns
import bokeh
import plotly
import sklearn
import tensorflow
import keras
from math import radians, cos, sin, asin, sqrt

Using TensorFlow backend.


## Business Understanding

## Data Understanding

### Data Upload
Data consists of several files which are uploaded

In [2]:
fs = s3fs.S3FileSystem(anon=True)

In [3]:
# **LIST THE AVAILABLE DATATHON CASES**
fs.ls('datacases/gd-2019/sofia_air_2')

['datacases/gd-2019/sofia_air_2/atmosphere_profile_train.csv',
 'datacases/gd-2019/sofia_air_2/construction_sites.csv',
 'datacases/gd-2019/sofia_air_2/household_heating.csv',
 'datacases/gd-2019/sofia_air_2/industrial_pollution.csv',
 'datacases/gd-2019/sofia_air_2/sofia-air-case2-test-set.zip',
 'datacases/gd-2019/sofia_air_2/sofia-air-case2.zip',
 'datacases/gd-2019/sofia_air_2/sofia_topo.csv',
 'datacases/gd-2019/sofia_air_2/stations_data_train.csv',
 'datacases/gd-2019/sofia_air_2/weather_lbsf_20161101-20161130_IP_train.csv']

In [4]:
with fs.open('datacases/gd-2019/sofia_air_2/atmosphere_profile_train.csv', 'rb') as f:
    df_atmosphere = pd.read_csv(f)
with fs.open('datacases/gd-2019/sofia_air_2/construction_sites.csv', 'rb') as f:
    df_construtions = pd.read_csv(f)
with fs.open('datacases/gd-2019/sofia_air_2/household_heating.csv', 'rb', encoding = "ISO-8859-1") as f:
    df_household = pd.read_csv(f)
with fs.open('datacases/gd-2019/sofia_air_2/sofia_topo.csv', 'rb', encoding = "ISO-8859-1") as f:
    df_sofiatopo = pd.read_csv(f)
with fs.open('datacases/gd-2019/sofia_air_2/stations_data_train.csv', 'rb', encoding = "ISO-8859-1") as f:
    df_stations = pd.read_csv(f)
with fs.open('datacases/gd-2019/sofia_air_2/weather_lbsf_20161101-20161130_IP_train.csv', 'rb', encoding = "ISO-8859-1") as f:
    df_weather = pd.read_csv(f)
# from data
df_stations_loc = pd.read_csv("Data/stations_locations.csv")
df_industrial = pd.read_csv("Data/industrial_pollution.csv", encoding = 'ISO-8859-1')

## Atmosphere and Dispersion Model

In [5]:
df_atmosphere.head()

,Date,HGHT(m),TEMP(C)
0,2016-11-01,595,9.6
1,2016-11-01,663,7.6
2,2016-11-01,844,5.4
3,2016-11-01,1047,3.6
4,2016-11-01,1284,1.5


Atmoshpere table contains height and temperature for given day and for corresponding height.
In the following code:
- we filter out only the values of HGHT(m) less than 2000 meters
- calculate the temperature in K
- calculate t/z value

In [6]:
df_atmosphere = df_atmosphere.loc[df_atmosphere['HGHT(m)'] < 2000]
df_atmosphere['TEMP(K)'] =  df_atmosphere['TEMP(C)'] + 273
df_atmosphere['t/z'] = df_atmosphere['TEMP(K)']/df_atmosphere['HGHT(m)']/100

In [7]:
# define the Atmospheric stability class with vertical temperature gradient
df_atmosphere.loc[df_atmosphere['t/z'] <= -1.9, 'PCS'] = 'A'
df_atmosphere.loc[df_atmosphere['t/z'].between(-1.9, 1.7), 'PCS'] = 'B'
df_atmosphere.loc[df_atmosphere['t/z'].between(-1.7, -1.5), 'PCS'] = 'C'
df_atmosphere.loc[df_atmosphere['t/z'].between(-1.5, 0.5), 'PCS'] = 'D'
df_atmosphere.loc[df_atmosphere['t/z'].between(-0.5, 1.5), 'PCS'] = 'E'
df_atmosphere.loc[df_atmosphere['t/z'].between(1.5, 4), 'PCS'] = 'F'
df_atmosphere.loc[df_atmosphere['t/z'] >= 4, 'PCS'] = 'G'

In [8]:
# σy and σx values based on Pasquill stability:
# if less than 1km and if > than 1km
df_atmosphere.loc[df_atmosphere['PCS'] == 'A', 'a'] = 213
df_atmosphere.loc[df_atmosphere['PCS'] == 'A', 'c<1km'] = 440.8
df_atmosphere.loc[df_atmosphere['PCS'] == 'A', 'd<1km'] = 1.941
df_atmosphere.loc[df_atmosphere['PCS'] == 'A', 'f<1km'] = 9.27
df_atmosphere.loc[df_atmosphere['PCS'] == 'A', 'c>1km'] = 459.7
df_atmosphere.loc[df_atmosphere['PCS'] == 'A', 'd>1km'] = 2.094
df_atmosphere.loc[df_atmosphere['PCS'] == 'A', 'f>1km'] = -9.6

df_atmosphere.loc[df_atmosphere['PCS'] == 'B', 'a'] = 156
df_atmosphere.loc[df_atmosphere['PCS'] == 'B', 'c<1km'] = 106.6
df_atmosphere.loc[df_atmosphere['PCS'] == 'B', 'd<1km'] = 1.149
df_atmosphere.loc[df_atmosphere['PCS'] == 'B', 'f<1km'] = 3.3
df_atmosphere.loc[df_atmosphere['PCS'] == 'B', 'c>1km'] = 108.2
df_atmosphere.loc[df_atmosphere['PCS'] == 'B', 'd>1km'] = 1.098
df_atmosphere.loc[df_atmosphere['PCS'] == 'B', 'f>1km'] = 2

df_atmosphere.loc[df_atmosphere['PCS'] == 'C', 'a'] = 104
df_atmosphere.loc[df_atmosphere['PCS'] == 'C', 'c<1km'] = 61
df_atmosphere.loc[df_atmosphere['PCS'] == 'C', 'd<1km'] = 0.911
df_atmosphere.loc[df_atmosphere['PCS'] == 'C', 'f<1km'] = 0
df_atmosphere.loc[df_atmosphere['PCS'] == 'C', 'c>1km'] = 61
df_atmosphere.loc[df_atmosphere['PCS'] == 'C', 'd>1km'] = 0.911
df_atmosphere.loc[df_atmosphere['PCS'] == 'C', 'f>1km'] = 0

df_atmosphere.loc[df_atmosphere['PCS'] == 'D', 'a'] = 68
df_atmosphere.loc[df_atmosphere['PCS'] == 'D', 'c<1km'] = 33.2
df_atmosphere.loc[df_atmosphere['PCS'] == 'D', 'd<1km'] = 0.725
df_atmosphere.loc[df_atmosphere['PCS'] == 'D', 'f<1km'] = -1.7
df_atmosphere.loc[df_atmosphere['PCS'] == 'D', 'c>1km'] = 44.5
df_atmosphere.loc[df_atmosphere['PCS'] == 'D', 'd>1km'] = 0.516
df_atmosphere.loc[df_atmosphere['PCS'] == 'D', 'f>1km'] = -13

df_atmosphere.loc[df_atmosphere['PCS'] == 'E', 'a'] = 50.5
df_atmosphere.loc[df_atmosphere['PCS'] == 'E', 'c<1km'] = 22.8
df_atmosphere.loc[df_atmosphere['PCS'] == 'E', 'd<1km'] = 0.678
df_atmosphere.loc[df_atmosphere['PCS'] == 'E', 'f<1km'] = -1.3
df_atmosphere.loc[df_atmosphere['PCS'] == 'E', 'c>1km'] = 55.4
df_atmosphere.loc[df_atmosphere['PCS'] == 'E', 'd>1km'] = 0.305
df_atmosphere.loc[df_atmosphere['PCS'] == 'E', 'f>1km'] = -34.0

df_atmosphere.loc[df_atmosphere['PCS'] == 'F', 'a'] = 34
df_atmosphere.loc[df_atmosphere['PCS'] == 'F', 'c<1km'] = 14.35
df_atmosphere.loc[df_atmosphere['PCS'] == 'F', 'd<1km'] = 0.740
df_atmosphere.loc[df_atmosphere['PCS'] == 'F', 'f<1km'] = -0.35
df_atmosphere.loc[df_atmosphere['PCS'] == 'F', 'c>1km'] = 62.6
df_atmosphere.loc[df_atmosphere['PCS'] == 'F', 'd>1km'] = 0.18
df_atmosphere.loc[df_atmosphere['PCS'] == 'F', 'f>1km'] = -48.6

In [9]:
# grouping buy most frequent class in a separate table and save as 'df_pcs' dataframe
df_pcs = df_atmosphere.groupby(['Date','PCS']).agg(lambda x:x.value_counts().index[0])
df_pcs.reset_index(level=df_pcs.index.names, inplace=True)
df_pcs['Date'] = df_pcs['Date'].str.strip()
df_pcs.head()

,Date,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,d<1km,f<1km,c>1km,d>1km,f>1km
0,2016-11-01,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
1,2016-11-02,E,1503,10.0,283.0,0.002257,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
2,2016-11-03,E,1487,13.2,280.0,0.003618,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
3,2016-11-04,E,1326,4.4,277.4,0.001808,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
4,2016-11-05,E,1279,5.3,281.7,0.001841,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0


The calculation of the stability class gives E class for all dates. We automatically pick the most frequent class in case it is different.

### Stations

In [10]:
df_stations.head()

,Date,STA-BG0052A,STA-BG0050A,STA-BG0073A,STA-BG0040A
0,2016-11-01,692.88,823.44,624.00,876.240
1,2016-11-02,1632.96,1756.56,1516.56,2382.288
2,2016-11-03,953.28,978.48,1086.00,680.736
3,2016-11-04,545.52,631.44,888.24,613.200
4,2016-11-05,1420.08,1664.40,1617.12,1608.480


In [11]:
#regroup the stations dataset
df_stations = pd.melt(df_stations, ['Date'], ['STA-BG0052A', 'STA-BG0050A', 'STA-BG0073A', 'STA-BG0040A'],
                      var_name='Station', value_name='PM10')
df_stations.head()

,Date,Station,PM10
0,2016-11-01,STA-BG0052A,692.88
1,2016-11-02,STA-BG0052A,1632.96
2,2016-11-03,STA-BG0052A,953.28
3,2016-11-04,STA-BG0052A,545.52
4,2016-11-05,STA-BG0052A,1420.08


In [12]:
df_stations.head()

,Date,Station,PM10
0,2016-11-01,STA-BG0052A,692.88
1,2016-11-02,STA-BG0052A,1632.96
2,2016-11-03,STA-BG0052A,953.28
3,2016-11-04,STA-BG0052A,545.52
4,2016-11-05,STA-BG0052A,1420.08


In [13]:
df_stations_merged = pd.merge(df_stations, df_pcs, how='inner', left_on='Date', right_on='Date')
df_stations_merged.head()

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,d<1km,f<1km,c>1km,d>1km,f>1km
0,2016-11-01,STA-BG0052A,692.88,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
1,2016-11-01,STA-BG0050A,823.44,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
2,2016-11-01,STA-BG0073A,624.00,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
3,2016-11-01,STA-BG0040A,876.24,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
4,2016-11-02,STA-BG0052A,1632.96,E,1503,10.0,283.0,0.002257,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0


In [14]:
#new feature day of week
df_stations_merged['Date'] = pd.to_datetime(df_stations_merged['Date'])
df_stations_merged['day_of_week'] = df_stations_merged['Date'].dt.day_name()
df_stations_merged.head()

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,d<1km,f<1km,c>1km,d>1km,f>1km,day_of_week
0,2016-11-01,STA-BG0052A,692.88,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
1,2016-11-01,STA-BG0050A,823.44,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
2,2016-11-01,STA-BG0073A,624.00,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
3,2016-11-01,STA-BG0040A,876.24,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
4,2016-11-02,STA-BG0052A,1632.96,E,1503,10.0,283.0,0.002257,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Wednesday


### Industrial distances

In [15]:
df_industrial.head()

,X*,Y*,m,t/y
0,"'""42°44''16.66""""N""'","'""23°14''28.82""""E""'",8.0,0.38
1,"'""42°39''46.01""""N""'","'""23°23''19.70""""E""'",15.0,0.03
2,"'""42°39''46.47""""N""'","'""23°23''19.27""""E""'",15.0,0.20
3,"'""42°39''46.70""""N""'","'""23°23''19.07""""E""'",15.0,0.96
4,"'""42°39''47.12""""N""'","'""23°23''21.30""""E""'",15.0,1.58


In [16]:
from math import cos, sqrt
fixedIndustrialDf = pd.read_csv("Data/industrial_coord.txt")

def get_distance(Lat1, Long1, Lat2, Long2):
    x = Lat2 - Lat1
    y = (Long2 - Long1)*cos((Lat2 + Lat1)*0.00872664626)  
    return 111.138*sqrt(x*x+y*y)



pd.options.mode.chained_assignment = None  # default='warn'

factoryDistanceDf = pd.DataFrame([['STA-BG0040A'], ['STA-BG0050A'], ['STA-BG0052A'], ['STA-BG0073A'] ], columns=['Name'])

for i in range(df_industrial.shape[0]):
    xcol = 'x' + str(i)
    factoryDistanceDf[xcol] = ''
    ycol = 'y' + str(i)
    factoryDistanceDf[ycol] = ''
    
    for j in range(factoryDistanceDf.shape[0]):
        stationLat = df_stations_loc.iloc[j]['Latitude']
        stationLon = df_stations_loc.iloc[j]['Longitude']
        
        factoryLat = fixedIndustrialDf.iloc[i]['Lat']
        factoryLon = fixedIndustrialDf.iloc[i]['Lon']
        
        distance = get_distance(stationLat, stationLon, factoryLat, factoryLon)
        
        #print(distance)
        factoryDistanceDf.loc[j][columnName] = distance

In [17]:
# calculated distances
factoryDistanceDf.head()

,Name,dist0,dist1,dist2,dist3,dist4,dist5,dist6,dist7,dist8,...,dist61,dist62,dist63,dist64,dist65,dist66,dist67,dist68,dist69,dist70
0,STA-BG0040A,5.71901,10.0049,9.98776,9.97939,10.0016,0.527694,10.9298,1.40481,1.36402,...,12.5022,10.5676,1.41387,1.45474,1.42651,5.07881,5.08076,5.08804,10.9448,10.9484
1,STA-BG0050A,7.82327,7.77454,7.76149,7.75531,7.80106,5.99211,16.6831,5.13246,5.13422,...,10.3761,16.3507,6.13188,6.16073,6.16291,8.44573,8.42728,8.44083,14.8085,14.8092
2,STA-BG0052A,15.2096,1.01649,1.01976,1.02119,0.969499,9.95558,17.3889,8.91983,8.96195,...,2.18718,17.2891,9.24499,9.22723,9.25564,15.1291,15.1232,15.1335,11.3041,11.2998
3,STA-BG0073A,7.8915,9.8703,9.85945,9.85437,9.90385,8.01768,18.6958,7.35019,7.34198,...,12.3495,18.3275,8.34323,8.37799,8.3743,8.94546,8.92123,8.93531,17.3711,17.3721


In [18]:
df_ind_model = pd.merge(df_stations_merged,factoryDistanceDf, how='left', left_on='Station', right_on='Name')
df_ind_model.columns

Index(['Date', 'Station', 'PM10', 'PCS', 'HGHT(m)', 'TEMP(C)', 'TEMP(K)',
       't/z', 'a', 'c<1km', 'd<1km', 'f<1km', 'c>1km', 'd>1km', 'f>1km',
       'day_of_week', 'Name', 'dist0', 'dist1', 'dist2', 'dist3', 'dist4',
       'dist5', 'dist6', 'dist7', 'dist8', 'dist9', 'dist10', 'dist11',
       'dist12', 'dist13', 'dist14', 'dist15', 'dist16', 'dist17', 'dist18',
       'dist19', 'dist20', 'dist21', 'dist22', 'dist23', 'dist24', 'dist25',
       'dist26', 'dist27', 'dist28', 'dist29', 'dist30', 'dist31', 'dist32',
       'dist33', 'dist34', 'dist35', 'dist36', 'dist37', 'dist38', 'dist39',
       'dist40', 'dist41', 'dist42', 'dist43', 'dist44', 'dist45', 'dist46',
       'dist47', 'dist48', 'dist49', 'dist50', 'dist51', 'dist52', 'dist53',
       'dist54', 'dist55', 'dist56', 'dist57', 'dist58', 'dist59', 'dist60',
       'dist61', 'dist62', 'dist63', 'dist64', 'dist65', 'dist66', 'dist67',
       'dist68', 'dist69', 'dist70'],
      dtype='object')

In [19]:
def calculateSigmaY(x, a):
    #sigmaY = a*x^0.894
    sigmaY = a * pow(x, 0.894)
    return sigmaY
 
def calculateSigmaZ(x, c, d, f):
    #sigmaZ = cx^(d+f)
    sigmaZ = c * pow(x, d+f)
    return sigmaZ
 
def calculateSigmaZFromRow(row, index):
    sigmaZ = calculateSigmaZ(row['dist' + str(index)], row['c>1km'], row['d>1km'], row['f>1km'])
    return sigmaZ
 
def calculateSigmaYFromRow(row, index):
    sigmaY = calculateSigmaY(row['dist' + str(index)], row['a'])
    return sigmaY
 
for index in range(0, df_industrial.shape[0]):
    df_ind_model['sigmaZ' + str(index)] = df_ind_model.apply(lambda row: calculateSigmaZFromRow(row, index), axis=1)
    df_ind_model['sigmaY' + str(index)] = df_ind_model.apply(lambda row: calculateSigmaYFromRow(row, index), axis=1)

In [20]:
df_ind_model.head()

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,...,sigmaZ66,sigmaY66,sigmaZ67,sigmaY67,sigmaZ68,sigmaY68,sigmaZ69,sigmaY69,sigmaZ70,sigmaY70
0,2016-11-01,STA-BG0052A,692.88,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,9.765111e-39,572.855504,9.894056e-39,572.656154,9.670846e-39,573.002956,1.797498e-34,441.454354,1.820899e-34,441.302881
1,2016-11-01,STA-BG0050A,823.44,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,3.314933e-30,340.176956,3.568363e-30,339.512691,3.380340e-30,340.000651,2.009902e-38,561.988369,2.006684e-38,562.012265
2,2016-11-01,STA-BG0073A,624.00,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,4.778057e-31,358.116504,5.235306e-31,357.249194,4.964515e-31,357.752953,9.279258e-41,648.182645,9.261918e-41,648.214813
3,2016-11-01,STA-BG0040A,876.24,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,9.180619e-23,215.894842,9.062184e-23,215.969232,8.635414e-23,216.245821,5.337785e-34,428.888363,5.279511e-34,429.013296
4,2016-11-02,STA-BG0052A,1632.96,E,1503,10.0,283.0,0.002257,50.5,22.8,...,9.765111e-39,572.855504,9.894056e-39,572.656154,9.670846e-39,573.002956,1.797498e-34,441.454354,1.820899e-34,441.302881


In [21]:
df_ind_model.columns

Index(['Date', 'Station', 'PM10', 'PCS', 'HGHT(m)', 'TEMP(C)', 'TEMP(K)',
       't/z', 'a', 'c<1km',
       ...
       'sigmaZ66', 'sigmaY66', 'sigmaZ67', 'sigmaY67', 'sigmaZ68', 'sigmaY68',
       'sigmaZ69', 'sigmaY69', 'sigmaZ70', 'sigmaY70'],
      dtype='object', length=230)

In [22]:
df_ind_model['PM10micro'] = df_ind_model['PM10'] / 1.225

In [23]:
# for indexheight in range(0,71):
#     df_ind_model['m'+str(indexheight)] = df_industrial.loc['m']

### Weather

In [24]:
df_weather.head()

,year,Month,day,TASMAX,TASAVG,TASMIN,DPMAX,DPAVG,DPMIN,RHMAX,...,sfcWindMAX,sfcWindAVG,sfcWindMIN,PSLMAX,PSLAVG,PSLMIN,PRCPMAX,PRCPAVG,PRCPMIN,VISIB
0,2016,11,1,12.7788,6.6672,0.0000,2.2224,-1.1112,-3.3336,87,...,24.14010,12.07005,0.0,1026.753044,1024.890530,1023.028016,-9999,0.00,-9999,5.471756
1,2016,11,2,15.5568,6.6672,-1.6668,2.7780,0.5556,-2.7780,100,...,11.26538,5.63269,0.0,1026.414405,1020.826864,1015.239322,-9999,0.00,-9999,8.046700
2,2016,11,3,13.3344,8.3340,3.3336,7.2228,3.3336,-1.1112,100,...,28.96812,14.48406,0.0,1023.366655,1019.133669,1014.900683,-9999,5.08,-9999,7.563898
3,2016,11,4,10.5564,6.1116,1.6668,6.1116,3.3336,1.1112,100,...,28.96812,14.48406,0.0,1025.398488,1023.197336,1020.996183,-9999,0.00,-9999,9.816974
4,2016,11,5,15.0012,8.8896,2.7780,10.0008,6.6672,2.7780,100,...,14.48406,7.24203,0.0,1024.043933,1020.657544,1017.271155,-9999,0.00,-9999,9.977908


In [25]:
def formatDateOfMonth(date):
    if len(str(date)) == 1:
        return '0'+ str(date)
    return date

df = df_weather
df['year'] = df['year'].astype(str)
df['Month'] = df['Month'].astype(str)
df['day'] = df['day'].apply(formatDateOfMonth)
df['day'] = df['day'].astype(str)

df['Date'] = df['year'].str.cat(df['Month'], sep='-')
df['Date'] = df['Date'].str.cat(df['day'], sep='-')#  +  + '-' + df['day']
df_weather = df

In [26]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 23 columns):
year          20 non-null object
Month         20 non-null object
day           20 non-null object
TASMAX        20 non-null float64
TASAVG        20 non-null float64
TASMIN        20 non-null float64
DPMAX         20 non-null float64
DPAVG         20 non-null float64
DPMIN         20 non-null float64
RHMAX         20 non-null int64
RHAVG         20 non-null float64
RHMIN         20 non-null int64
sfcWindMAX    20 non-null float64
sfcWindAVG    20 non-null float64
sfcWindMIN    20 non-null float64
PSLMAX        20 non-null float64
PSLAVG        20 non-null float64
PSLMIN        20 non-null float64
PRCPMAX       20 non-null int64
PRCPAVG       20 non-null float64
PRCPMIN       20 non-null int64
VISIB         20 non-null float64
Date          20 non-null object
dtypes: float64(15), int64(4), object(4)
memory usage: 3.7+ KB


In [27]:
df_industrial.head()

,X*,Y*,m,t/y
0,"'""42°44''16.66""""N""'","'""23°14''28.82""""E""'",8.0,0.38
1,"'""42°39''46.01""""N""'","'""23°23''19.70""""E""'",15.0,0.03
2,"'""42°39''46.47""""N""'","'""23°23''19.27""""E""'",15.0,0.20
3,"'""42°39''46.70""""N""'","'""23°23''19.07""""E""'",15.0,0.96
4,"'""42°39''47.12""""N""'","'""23°23''21.30""""E""'",15.0,1.58


### Final Merge

In [28]:
df_ind_model['Date'] = df_ind_model['Date'].astype(str)
df_ind_model['Date'] = df_ind_model['Date'].astype(str)

In [29]:
df_ind_model = pd.merge(df_ind_model,df_weather, how='left', left_on='Date', right_on='Date')
df_ind_model

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,...,sfcWindMAX,sfcWindAVG,sfcWindMIN,PSLMAX,PSLAVG,PSLMIN,PRCPMAX,PRCPAVG,PRCPMIN,VISIB
0,2016-11-01,STA-BG0052A,692.880,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
1,2016-11-01,STA-BG0050A,823.440,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
2,2016-11-01,STA-BG0073A,624.000,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
3,2016-11-01,STA-BG0040A,876.240,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
4,2016-11-02,STA-BG0052A,1632.960,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
5,2016-11-02,STA-BG0050A,1756.560,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
6,2016-11-02,STA-BG0073A,1516.560,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
7,2016-11-02,STA-BG0040A,2382.288,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
8,2016-11-03,STA-BG0052A,953.280,E,1487,13.2,280.0,0.003618,50.5,22.8,...,28.96812,14.48406,0.00000,1023.366655,1019.133669,1014.900683,-9999,5.080,-9999,7.563898
9,2016-11-03,STA-BG0050A,978.480,E,1487,13.2,280.0,0.003618,50.5,22.8,...,28.96812,14.48406,0.00000,1023.366655,1019.133669,1014.900683,-9999,5.080,-9999,7.563898


In [30]:
list(df_ind_model.columns)

['Date',
 'Station',
 'PM10',
 'PCS',
 'HGHT(m)',
 'TEMP(C)',
 'TEMP(K)',
 't/z',
 'a',
 'c<1km',
 'd<1km',
 'f<1km',
 'c>1km',
 'd>1km',
 'f>1km',
 'day_of_week',
 'Name',
 'dist0',
 'dist1',
 'dist2',
 'dist3',
 'dist4',
 'dist5',
 'dist6',
 'dist7',
 'dist8',
 'dist9',
 'dist10',
 'dist11',
 'dist12',
 'dist13',
 'dist14',
 'dist15',
 'dist16',
 'dist17',
 'dist18',
 'dist19',
 'dist20',
 'dist21',
 'dist22',
 'dist23',
 'dist24',
 'dist25',
 'dist26',
 'dist27',
 'dist28',
 'dist29',
 'dist30',
 'dist31',
 'dist32',
 'dist33',
 'dist34',
 'dist35',
 'dist36',
 'dist37',
 'dist38',
 'dist39',
 'dist40',
 'dist41',
 'dist42',
 'dist43',
 'dist44',
 'dist45',
 'dist46',
 'dist47',
 'dist48',
 'dist49',
 'dist50',
 'dist51',
 'dist52',
 'dist53',
 'dist54',
 'dist55',
 'dist56',
 'dist57',
 'dist58',
 'dist59',
 'dist60',
 'dist61',
 'dist62',
 'dist63',
 'dist64',
 'dist65',
 'dist66',
 'dist67',
 'dist68',
 'dist69',
 'dist70',
 'sigmaZ0',
 'sigmaY0',
 'sigmaZ1',
 'sigmaY1',
 'sigmaZ

## Define and calculate C(x,y,z)

In [49]:
def C_xyz(Q,u,sigmaY,sigmaZ,h,z,y):
#     Cxyz = ((np.e**((-(z-h)**2)/(2*sigmaZ**2)))+(np.e**((-(z+h)**2)/(2*sigmaZ**2))))
    Cxyz = (Q/(2*np.pi*u*sigmaY*sigmaZ))*(np.e**((y**2)/(2*sigmaY**2)))*((np.e**((-(z-h)**2)/(2*sigmaZ**2)))*(np.e**((-(z+h)**2)/(2*sigmaZ**2))))
    return Cxyz

In [32]:
# list(df_ind_model.columns)

In [33]:
df_ind_model['Ctotal'] = 0

#### Assumptions:
- y=x becuase the pollutions spreads radially
- z = 0, becuase of the ground level
- h is the height of the chimney

In [47]:
def calculateCxyz(row, index):
    Cxyz = C_xyz(Q=row['PM10micro'],
                 u=row['sfcWindAVG'],
#                  sigmaY = 300,
                 sigmaY = row['sigmaY' + str(index)], 
#                  sigmaZ = 300,
                 sigmaZ = row['sigmaZ' + str(index)],
                 h = df_industrial.loc[index]['m'],
                 z = 0, 
                 y = row['dist' + str(index)])
    return Cxyz

df_ind_model['Ctotal'] = 0
 
for index in range(0, df_industrial.shape[0]):
    df_ind_model['C' + str(index)] = df_ind_model.apply(lambda row: calculateCxyz(row, index), axis=1)
    df_ind_model['Ctotal'] = df_ind_model['Ctotal'] + df_ind_model['C' + str(index)] 


In [48]:
np.sum(df_ind_model.Ctotal.head())

0.16969617122052857

In [36]:
print(np.exp)

<ufunc 'exp'>


In [37]:
df_construtions.head()

,id,start date,type,district,locality,address
0,100,07.1.2016,non-residential,OVCHA KUPEL,SEKULITSA,SUHOL
1,101,11.1.2016,non-residential,KRASNA POLYANA,TRUDOVI KAZARMI,street SUHOLSKA
2,102,11.1.2016,non-residential,MLADOST,MLADOST 2,NaN
3,103,13.1.2016,infrastructure,ISKAR,NADEZHDA 2A 2B,NaN
4,104,13.1.2016,infrastructure,NADEZHDA,STANKE DIMITROV,NaN


In [38]:
df_household.head()

,X,Y,NJ16_eq_1,NJ16_eq_2,NJ16_eq_3,NJ17_eq_1,NJ17_eq_3,NJ17_eq_4,NJ17_eq_6,NJ17_eq_7,NJ17_eq_4i,NJ17_eq_8,NJ17_eq_9,NN_Jilisht,NBROI_LICA
0,23.383978,42.693180,NaN,NaN,1.0,NaN,NaN,0,1,0,1,NaN,NaN,1,4
1,23.383978,42.693180,NaN,NaN,1.0,NaN,NaN,0,1,0,1,NaN,NaN,1,2
2,23.382120,42.693912,NaN,1.0,NaN,NaN,NaN,0,1,0,1,NaN,NaN,1,4
3,23.376602,42.678282,NaN,NaN,1.0,NaN,NaN,0,0,1,1,NaN,NaN,1,4
4,23.383978,42.693180,NaN,NaN,1.0,NaN,NaN,0,1,0,1,NaN,NaN,1,1


In [39]:
df_industrial.head()

,X*,Y*,m,t/y
0,"'""42°44''16.66""""N""'","'""23°14''28.82""""E""'",8.0,0.38
1,"'""42°39''46.01""""N""'","'""23°23''19.70""""E""'",15.0,0.03
2,"'""42°39''46.47""""N""'","'""23°23''19.27""""E""'",15.0,0.20
3,"'""42°39''46.70""""N""'","'""23°23''19.07""""E""'",15.0,0.96
4,"'""42°39''47.12""""N""'","'""23°23''21.30""""E""'",15.0,1.58


In [40]:
df_sofiatopo.head()

,Lat,Lon,Elev
0,42.62,23.220000,1184.0
1,42.62,23.233571,1333.0
2,42.62,23.247143,1505.0
3,42.62,23.260714,1586.0
4,42.62,23.274286,1533.0


In [41]:
df_stations.head()

,Date,Station,PM10
0,2016-11-01,STA-BG0052A,692.88
1,2016-11-02,STA-BG0052A,1632.96
2,2016-11-03,STA-BG0052A,953.28
3,2016-11-04,STA-BG0052A,545.52
4,2016-11-05,STA-BG0052A,1420.08


In [42]:
df_stations_loc.head()

,AirQualityStationEoICode,CommonName,Longitude,Latitude
0,BG0040A,Nadezhda,23.310972,42.732292
1,BG0050A,Hipodruma,23.296786,42.680558
2,BG0052A,Druzhba,23.400164,42.666508
3,BG0073A,IAOS/Pavlovo,23.268403,42.669797


**GUIDELINE HOW TO READ THE TELENOR DATA IN PANDAS DATAFRAME**

In [43]:
import zipfile
fs.get('datacases/datathon-2018-2/CASE-NAME/FILE-NAME.csv', 'local_FILE_NAME.zip')
zf = zipfile.ZipFile('data.zip') 
df = pandas.read_csv(zf.open('data.csv'))

FileNotFoundError: datacases/datathon-2018-2/CASE-NAME/FILE-NAME.csv

**LIST FILES IN THE DIR**

In [ ]:
!ls